# Target Accrual Redemption Forward (TARF) Target KO

This notebook demonstrates how to access and use the functionalities of **Target Accrual Redemption Forward (TARF)** which are part of our **QPS** module within LSEG Financial Analytics SDK.

**You will be able to:**
* Define TARF instrument with basic parameters
* Configure pricing parameters
* Evaluate TARF Analytics

A TARF target KO is a TARF (possibly with EKI) with an additional KO provision observed only on fixing dates.

The KO direction is determined by the Call/Put side, for a Call (Put) the KO is Above (Below). When, on a fixing date, the spot is observed through the KO level, the holder is entitled to be paid immediately the residual difference between the target and the current cumulated profit. The instrument is also redeemed.

The Target KO condition prevails (in the sense that is tested before) the regular termination condition based on reaching the target.

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp

import datetime as dt
import pandas as pd
import json
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example target KO TARF, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (underlying rate, payoff description, dates, notional)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing models and parameters, optional

In [2]:
# 1. Create SP definition object

print("Step 1: Configuring instrument definition...")

TARF_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "TARF_Target_KO",
    inputs = [
        sp.NameTypeValue(name="Underlying", type = "string", value="USDEUR"),
        sp.NameTypeValue(name="StartDate", type = "date", value= dt.date(2022, 1, 25)),
        sp.NameTypeValue(name="EndDate", type = "string", value="StartDate + 1Y"),
        sp.NameTypeValue(name="Frequency", type = "string", value="3M"),
        sp.NameTypeValue(name="Notional", type = "string", value="100000"),
        sp.NameTypeValue(name="Strike", type = "string", value="0.8"),
        sp.NameTypeValue(name="KnockOutRate", type = "string", value="1.02"),
        sp.NameTypeValue(name="IsKnockOutEvent", type = "string", value="FX[t] >= KnockOutRate"),
        sp.NameTypeValue(name="IsRedemptionEvent", type = "string", value="Sum[t] >= ProfitTarget"),
        sp.NameTypeValue(name="ProfitTarget", type = "string", value="4%"),
        sp.NameTypeValue(name="ResidualProfitTarget", type = "string", value="Max(ProfitTarget - Sum[t-1], 0)"),
        sp.NameTypeValue(name="KO_Payment", type = "string", value="Settlement[t]"),
    ],
    payoff_description = [
					[
                        "Schedule Type",
                        "Schedule description",
                        "FX",
                        "Coupon",
                        "Settlement",
                        "Sum",
                        "Alive",
                        "KO_Amount",
                        "Price"
                    ],
                    [
                        "AtDate",
                        "StartDate",
                        "",
                        "",
                        "0",
                        "",
                        "IF(ProfitTarget >= 0, 1, 0)",
                        "",
                        ""
                    ],
                    [
                        "OnSchedule",
                        "DateTable(StartDate + Frequency, EndDate, Frequency, ResetGap := 0b)",
                        "FxSpot(Underlying)",
                        "IF(IsKnockOutEvent, ResidualProfitTarget, FX[t] - Strike)",
                        "Coupon[t] * Notional",
                        "Sum[LastDate] + max(Coupon[t],0)",
                        "If(IsRedemptionEvent or IsKnockOutEvent, 0, Alive[LastDate-1])",
                        "Alive[LastDate-1] * (1-Alive[LastDate]) * KO_Payment",
                        "Receive (Alive[t] * Settlement[t] + KO_Amount[t])"
                    ],
                    [
                        "AtDate",
                        "EndDate",
                        "FxSpot(Underlying)",
                        "IF(IsKnockOutEvent, ResidualProfitTarget, FX[t] - Strike)",
                        "Coupon[LastDate] * Notional",
                        "Sum[LastDate] + max(Coupon[t],0)",
                        "If(IsRedemptionEvent or IsKnockOutEvent, 0, Alive[LastDate-1])",
                        "Alive[LastDate-1] * (1-Alive[LastDate]) * KO_Payment",
                        "Receive (Alive[LastDate] * Settlement[t] + KO_Amount[t])"
                    ]
				]
)


print("	Instrument definition configured")

# 2. Create SP instrument definition object
print("Step 2: Creating instrument definition object...")

tarf_target_ko = sp.StructuredProductsDefinitionInstrument(definition = TARF_definition)
print("	Instrument definition created")


# 3. Create SP parameters object - optional
print("Step 3: Configuring pricing parameters...")

TARF_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2022, 3, 16),  # Set your desired valuation date
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "USDEUR",
            underlying_tag = "USDEUR",
            underlying_currency = "EUR",
            asset_class = "ForeignExchange",
            model_name= "Heston")]
)
print("	Pricing parameters configured")

Step 1: Configuring instrument definition...
	Instrument definition configured
Step 2: Creating instrument definition object...
	Instrument definition created
Step 3: Configuring pricing parameters...
	Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price function
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = sp.price(
        definitions=[tarf_target_ko],
        pricing_preferences=TARF_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing Execution Successful!")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing Execution Successful!


## Results Display

### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., underlying, payoff description, dates, notional), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method, it is StructuredProductsPricingParameters we used

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` with discount factors
   - **description**: Instrument summary StructuredProductDefinition and also default fields not specified in the StructuredProductDefinition, but used by default in the calculation (e.g. price)
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

The analytics fields are useful for understanding which fields are included by default in the price function, even if they are not explicitly specified in `price` function. We detail below the description, valuation, cash-flows and greeks.

### Description

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "TARF_Target_KO",
    "dealCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 0.040873302,
        "Coupon": 0.361995511,
        "FX": 3.603602536,
        "KO_Amount": 10459.966529269,
        "Price": 10411.850025818,
        "Settlement": 36199.551141786,
        "Sum": 0.961770256
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_tarf_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_tarf_valuation)

,Field,Value
0,marketValueInDealCcy,10411.850026


### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Build dataframes for all cash flow types
df_fx = pd.DataFrame(cashflows[0]['payments']).rename(columns={'amount': cashflows[0]['legTag']})
df_coupon = pd.DataFrame(cashflows[1]['payments']).rename(columns={'amount': cashflows[1]['legTag']})
df_settlement = pd.DataFrame(cashflows[2]['payments']).rename(columns={'amount': cashflows[2]['legTag']})
df_sum  = pd.DataFrame(cashflows[3]['payments']).rename(columns={'amount': cashflows[3]['legTag']})
df_alive = pd.DataFrame(cashflows[4]['payments']).rename(columns={'amount': cashflows[4]['legTag']})
df_ko_amount = pd.DataFrame(cashflows[5]['payments']).rename(columns={'amount': cashflows[5]['legTag']})
df_price = pd.DataFrame(cashflows[6]['payments']).rename(columns={'amount': cashflows[6]['legTag']})

# Merge all dataframes on the 'date' column
# Add suffix to distinguish between different data sources
combined_df = pd.concat(
    [df_fx.add_suffix('_fx'), 
     df_coupon.add_suffix('_coupon'), 
     df_settlement.add_suffix('_settlement'), 
     df_sum.add_suffix('_sum'), 
     df_alive.add_suffix('_alive'), 
     df_ko_amount.add_suffix('_ko_amount'), 
     df_price.add_suffix('_price')], 
    axis=1
)

# Remove duplicate columns with the same values
combined_df = combined_df.loc[:, ~combined_df.T.duplicated()]

# Keep only one discountFactor column and sort the columns
combined_df = combined_df.loc[:,['date_fx', 'discountFactor_fx', 'FX_fx', 'Coupon_coupon','Settlement_settlement','Sum_sum',  'Alive_alive',
       'KO_Amount_ko_amount','Price_price', 'currency_fx', 'event_fx','occurence_fx']]

# Rename columns to match the desired layout
combined_df.columns = ['date', 'discountFactor', 'FX', 'Coupon', 'Settlement', 'Sum', 'Alive', 'KO_Amount', 'Price', 'currency', 'event', 'occurrence']

# Display the combined dataframe
display(combined_df)

,date,discountFactor,FX,Coupon,Settlement,Sum,Alive,KO_Amount,Price,currency,event,occurrence
0,2022-01-25,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,EUR,Payoff,Historical
1,2022-04-25,1.000636,0.905213,0.104194,10419.352313,0.104214,0.0196,10375.487715,10434.014530,EUR,Payoff,Future
2,2022-07-25,1.002044,0.901621,0.093902,9390.179958,0.199054,0.0078,68.206506,45.348684,EUR,Payoff,Future
3,2022-10-25,1.003093,0.896887,0.085769,8576.884288,0.287300,0.0068,8.230344,-28.885502,EUR,Payoff,Future
4,2023-01-25,1.003616,0.891465,0.077328,7732.827603,0.368508,0.0066,1.275608,-45.102037,EUR,Payoff,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_tarf_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_tarf_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,866.87000
1,gammaAmountInDealCcy,-35.08360
2,vegaAmountInDealCcy,-219.37300
3,thetaAmountInDealCcy,1.20278
